# 콘텍스트 관리자와 else 블록

## 이것 다음에 저것: if 문 이외에서의 else 블록

In [ ]:
for item in my_list:
    if item.flavor == 'banana':
        break
# break로 빠져나오지 않으면 해당 블록 실행
else:
    raise ValueError("No banana flavor found!")

In [ ]:
try:
    dangerous_call()
    after_call() # after_call을 try 안에 넣는 것은 좋아 보이지 않는다.
except OSError:
    log('OSError...')

In [ ]:
try:    # try 블록 안에는 예외를 발생시킬 가능성이 있는 코드만 넣어야 한다.
    dangerous_call() 
except OSError:
    log('OSError...')
else:   # 코드의 의도를 명확히 하기 위해 else문 사용.
    after_call()    

#### EAFP   
Easier to Ask for Forgiveness than Permission   
허락을 구하기보다 용서를 구하는 것이 더 쉽다.


#### LBYL   
Leap Before You Leap   
누울 자리를 보고 다리를 뻗으라.

## 콘텍스트 관지라와 with 블록
반복자가 for 문을 제어하기 위해 존재하는 것과 마찬가지로,   
콘텍스트 관지라 객체는 with 문을 제어하기 위해 존재한다.

In [ ]:
# 파일의 __enter__() 메서드가 self를 반환하므로 fp는 열린 파일에 바인딩 된다.
with open('mirror.py') as fp:
    # fp에서 데이터를 읽는다.
    src = fp.read(60)

len(src)

In [ ]:
''' 
open() 함수가 TextIOWrapper 객체를 반환하고 이 객체의 __enter__() 메서드는 self을 반환하므로
TextIOWrapper 객체가 as 절에 있는 타겟 변수는 fp에 바인딩 된다.
'''
fp # fp 변수는 여전히 살아있다.

In [ ]:
fp.closed, fp.encoding

In [ ]:
fp.read(60) 
# with block이 끝날 때 TextIOWrapper.__exit__() 메서드가 호출되어 파일을 닫으므로,
# fp를 이용해서 파일 입출력을 더이상 할 수 없다.

In [ ]:
from mirror import LookingGlass
with LookingGlass() as what: # LookingGlass 객체는 콘텍스트 관리자다.
    print('Alice, Kitty and Snowdrop')
    # print(4//0)
    print(what)
    

In [ ]:
what

In [ ]:
print('back to normal')

#### \_\_enter__() 메서를 호출할 때는 다음 세 인수를 전달한다.   
exc_type   
ZeroDivisionError 등의 예외 클래스

exc_value   
예외 객체.

traceback   
traceback 객체

In [ ]:
from mirror import LookingGlass
manager = LookingGlass()
manager

In [ ]:
monster = manager.__enter__()

In [ ]:

monster == 'JABBERWOCKY'

In [ ]:
monster

In [ ]:
manager

In [ ]:
manager.__exit__(None, None, None)

In [ ]:
monster

## contextlib 유틸리티

### @contextmanager
클래스를 생성하고 프로토콜을 구현하는 대신,   
간단한 제너레이터 함수로부터 컨텍스트 관지라를 생성할 수 있게 해주는 데커레이터.

In [ ]:
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_wrtie(text):
        return original_write(text[::-1])
    sys.stdout.write = reverse_wrtie
    yield 'JABBERWOCKY'
    sys.stdout.write = original_write

In [ ]:
from mirror_gen import looking_glass
with looking_glass() as what: # LookingGlass 객체는 콘텍스트 관리자다.
    print('Alice, Kitty and Snowdrop')
    print(what)
    

#### in-place file rewriting context manager

In [ ]:
from contextlib import contextmanager
import io
import os


@contextmanager
def inplace(filename, mode='r', buffering=-1, encoding=None, errors=None,
            newline=None, backup_extension=None):
    """Allow for a file to be replaced with new content.

    yields a tuple of (readable, writable) file objects, where writable
    replaces readable.

    If an exception occurs, the old file is restored, removing the
    written data.

    mode should *not* use 'w', 'a' or '+'; only read-only-modes are supported.

    """

    # move existing file to backup, create new file with same permissions
    # borrowed extensively from the fileinput module
    if set(mode).intersection('wa+'):
        raise ValueError('Only read-only file modes can be used')

    backupfilename = filename + (backup_extension or os.extsep + 'bak')
    try:
        os.unlink(backupfilename)
    except os.error:
        pass
    os.rename(filename, backupfilename)
    readable = io.open(backupfilename, mode, buffering=buffering,
                       encoding=encoding, errors=errors, newline=newline)
    try:
        perm = os.fstat(readable.fileno()).st_mode
    except OSError:
        writable = open(filename, 'w' + mode.replace('r', ''),
                        buffering=buffering, encoding=encoding, errors=errors,
                        newline=newline)
    else:
        os_mode = os.O_CREAT | os.O_WRONLY | os.O_TRUNC
        if hasattr(os, 'O_BINARY'):
            os_mode |= os.O_BINARY
        fd = os.open(filename, os_mode, perm)
        writable = io.open(fd, "w" + mode.replace('r', ''), buffering=buffering,
                           encoding=encoding, errors=errors, newline=newline)
        try:
            if hasattr(os, 'chmod'):
                os.chmod(filename, perm)
        except OSError:
            pass
    try:
        yield readable, writable
    except Exception:
        # move backup back
        try:
            os.unlink(filename)
        except os.error:
            pass
        os.rename(backupfilename, filename)
        raise
    finally:
        readable.close()
        writable.close()
        try:
            os.unlink(backupfilename)
        except os.error:
            pass

In [ ]:
import csv

csvfilename = 'csvfile.csv'
with inplace(csvfilename, 'r', newline='\n') as (infh, outfh):
    reader = csv.reader(infh)
    writer = csv.writer(infh)

    for row in reader:
        row += ['new', 'columns']
        writer.writerow(row)
print(1)